In [1]:
import glob
import pandas as pd
import json
from tqdm import tqdm
from ipa.src.worker import convert
from collections import defaultdict

In [2]:
audio_paths = glob.glob('./data/korean/*/*/*.pcm')
len(audio_paths)

628545

In [3]:
label_paths = glob.glob('./data/korean/*/*/*.txt')
len(label_paths)

622545

In [4]:
df_list = defaultdict(list)
error_count = 0
for file in tqdm(label_paths):
    try:
        with open(file, 'r', encoding='euc-kr') as f:
            data = f.read()
    except:
        error_count += 1
        continue
    df_list['audio_path'].append(file[:-3] + "pcm")
    df_list['label_path'].append(file)
    df_list['label'].append(data)
merged_df = pd.DataFrame(df_list)
print(f'total len: {len(merged_df)}\terror count: {error_count}')

100%|██████████| 622545/622545 [00:06<00:00, 92956.40it/s]


total len: 620800	error count: 1745


In [5]:
merged_df.head()

,audio_path,label_path,label
0,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,그렇게 되면 또 (2월)/(이 월)로 넘어가겠지.\n
1,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,뭐/ 기아. 막/ 그런 사람들이 어떻게 더 좋게 살아갈 수 있을까. 이렇게 b/ 평...
2,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,"중점적으로 하는 학원인데, b/ 그거를 해서 아이들이 공부하는 까. b/\n"
3,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,공단은 나도 잘 모르겠다. (4층)/(사 층)인가? n/ 아니 공단.\n
4,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,n/ 아까 뭐 만들었는지 얘기해줬잖아.\n


In [6]:
merged_df.to_pickle('./data/korean/merged.pickle')

In [7]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620800 entries, 0 to 620799
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   audio_path  620800 non-null  object
 1   label_path  620800 non-null  object
 2   label       620800 non-null  object
dtypes: object(3)
memory usage: 14.2+ MB


In [8]:
import re
from collections import Counter

In [9]:
def count_special_characters(text):
    """
    주어진 텍스트에서 특수문자(알파벳, 숫자, 밑줄, 공백 제외)를 찾아 개수를 Counter 객체로 반환합니다.
    """
    # [^\w\s] : \w (단어문자; 유니코드 모드에서는 한글도 포함)와 \s (공백 문자)를 제외한 모든 문자
    specials = re.findall(r'[^\w\s]', text)
    return Counter(specials)

def count_special_characters_in_dataframe(df, text_column='text'):
    """
    DataFrame의 지정된 text_column에 있는 모든 텍스트에서 특수문자를 추출하여,
    전체 특수문자 종류와 개수를 딕셔너리 형태로 반환합니다.
    """
    total_counts = Counter()
    # 결측치를 제외하고 순회
    for text in tqdm(df[text_column].dropna()):
        total_counts += count_special_characters(text)
    return dict(total_counts)

In [10]:
special_char_counts = count_special_characters_in_dataframe(merged_df, text_column='label')
print("특수문자 개수 집계:")
for char, count in special_char_counts.items():
    print(f"'{char}': {count}")

100%|██████████| 620800/620800 [00:01<00:00, 395536.20it/s]

특수문자 개수 집계:
'(': 177520
')': 177520
'/': 971355
'.': 638976
'+': 111375
',': 152887
'?': 234258
'*': 65333
'%': 475
'!': 838
'-': 29
'&': 1
':': 16
'#': 1


In [11]:
import random
spec_to_sentences = defaultdict(list)

for text in tqdm(merged_df['label'].dropna()):
    specs = re.findall(r'[^\w\s]', text)
    # 같은 문장 내에서도 중복된 특수문자 대신 한번만 기록하도록 set 사용
    for spec in set(specs):
        spec_to_sentences[spec].append(text)

result = {}
for spec, sentences in spec_to_sentences.items():
    result[spec] = random.choice(sentences)
    
for spec, sentence in result.items():
    print(f"특수문자 '{spec}'가 포함된 문장: {sentence}")

100%|██████████| 620800/620800 [00:00<00:00, 669604.47it/s]

특수문자 '.'가 포함된 문장: o/ 유럽.

특수문자 '/'가 포함된 문장: 자파해*? l/

특수문자 '('가 포함된 문장: 그러면 (1월)/(일 월) 후반쯤에 가야겠네? n/

특수문자 ')'가 포함된 문장: o/ 아/ 나는 이투스 (2 4 7)/(이 사 칠)에 다니다 보니까 이투스 그게 꽁짜여가지고 b/

특수문자 '+'가 포함된 문장: 요+ 맞다. 그어 딱 느꼈어. 한+ 하+ 한마디로 요약을 하면은 이젠 데이터에 시대다.

특수문자 ','가 포함된 문장: b/ 야, l/ 그/ b/ 음악쌤 기억나? 멸치.

특수문자 '?'가 포함된 문장: 왜?

특수문자 '*'가 포함된 문장: n/ 그이까* 그/ 일본 갔을 때처럼 b/ 쪼그만 거 하나 뽑힐 거 같다고 (3만 원)/(삼만 원) 넣는 그런 행동만 하지 말자고*

특수문자 '%'가 포함된 문장: b/ 그래서 아직 손발이 (100%)/(백 퍼센트) 안 맞는 느낌이었어.

특수문자 '!'가 포함된 문장: o/ 아! 근데 좀 도움 되던데? 콤보 연계 이런 거 알려 주잖아.

특수문자 '-'가 포함된 문장: 보통 주안역에서 내려서 (515-1)/(오백 십 오 다시 일)이나 (515번)/(오백 십 오 번)을 타고 가면은 그 앞에서 내려 주거든 원래 그렇게 갔는데

특수문자 '&'가 포함된 문장: 응. 그 만화영화인데 이제 아동극이지. 그거 b/ 삼성역 앞에 상상 아트홀 (KT&G)/(케이 티 엔 지) 거기서 했는데. b/

특수문자 ':'가 포함된 문장: b/ 그러다 보니까 뭐/ 굳이 뭐/ (9:1)/(구 대 일)? (8:2)/(팔 대 이) 이런 비율을 나눈다기보다 b/ 뭐/ 연애를 할 때 있어서 항상 내가 더 내곤 했지. b/ 근데 이제 b/

특수문자 '#'가 포함된 문장: #



In [12]:
def transform_func(text: str) -> str:
    pattern = r'\((.*?)\)/\((.*?)\)'
    transformed_text = text.replace('\n', '')
    transformed_text = re.sub(pattern, r'\2', transformed_text)
    # transformed_text = re.sub(r"[:'`]", "", transformed_text)
    # transformed_text = re.sub(r"[·]", " ", transformed_text)
    transformed_text = re.sub(r'(?:b/|l/|o/|n/|[\/+*\.,?!]|#)', '', transformed_text)
    
    pattern = re.compile(r'\s\s+') # 스페이스바 두 번 이상일 때
    transformed_text = re.sub(pattern, ' ', transformed_text.strip())
    return transformed_text

In [13]:
transform_func("아/ 고게 꽤 큰가 보네. 죽전, (4D)/(포 D)까지 하는 거면")

'아 고게 꽤 큰가 보네 죽전 포 D까지 하는 거면'

In [14]:
merged_df['label.transformed'] = merged_df['label'].apply(transform_func)

In [15]:
special_char_counts = count_special_characters_in_dataframe(merged_df, text_column='label.transformed')
print("특수문자 개수 집계:")
for char, count in special_char_counts.items():
    print(f"'{char}': {count}")

100%|██████████| 620800/620800 [00:00<00:00, 647380.34it/s]

특수문자 개수 집계:
'%': 9
'-': 15


In [16]:
import random
spec_to_sentences = defaultdict(list)

for text in tqdm(merged_df['label.transformed'].dropna()):
    specs = re.findall(r'[^\w\s]', text)
    # 같은 문장 내에서도 중복된 특수문자 대신 한번만 기록하도록 set 사용
    for spec in set(specs):
        spec_to_sentences[spec].append(text)

result = {}
for spec, sentences in spec_to_sentences.items():
    result[spec] = random.choice(sentences)
    
for spec, sentence in result.items():
    print(f"특수문자 '{spec}'가 포함된 문장: {sentence}")

  0%|          | 0/620800 [00:00<?, ?it/s]

100%|██████████| 620800/620800 [00:00<00:00, 1159565.96it/s]

특수문자 '%'가 포함된 문장: 그리고 너가 누굴 데리고 오잖아 그럼 그 %를 너도 받고 나도 받아 왜냐면 내가 데려온 애고
특수문자 '-'가 포함된 문장: 아 너가 보기에는 안 좋을 거야 너가 맨날 그 x-ray 로 막 그거 보니까 내 거 x-ray 하면 너 완전 헤 할걸


In [17]:
filtered_df = merged_df[merged_df['label.transformed'].str.match(r'^[ㄱ-ㅎ가-힣 ]+$')]

In [18]:
filtered_df.head()

,audio_path,label_path,label,label.transformed
0,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,그렇게 되면 또 (2월)/(이 월)로 넘어가겠지.\n,그렇게 되면 또 이 월로 넘어가겠지
1,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,뭐/ 기아. 막/ 그런 사람들이 어떻게 더 좋게 살아갈 수 있을까. 이렇게 b/ 평...,뭐 기아 막 그런 사람들이 어떻게 더 좋게 살아갈 수 있을까 이렇게 평상시에도 관심...
2,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,"중점적으로 하는 학원인데, b/ 그거를 해서 아이들이 공부하는 까. b/\n",중점적으로 하는 학원인데 그거를 해서 아이들이 공부하는 까
3,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,공단은 나도 잘 모르겠다. (4층)/(사 층)인가? n/ 아니 공단.\n,공단은 나도 잘 모르겠다 사 층인가 아니 공단
4,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,n/ 아까 뭐 만들었는지 얘기해줬잖아.\n,아까 뭐 만들었는지 얘기해줬잖아


In [19]:
filtered_df.shape

(603978, 4)

In [20]:
def convert_ipa(item):
    return convert(item, rules_to_apply='pastcnovr')['result_array']

In [21]:
res = []
res_list = []
for v in tqdm(filtered_df['label.transformed']):
    con = convert(v, rules_to_apply='pastcnovr')
    res.append(con['result'])
    res_list.append(con['result_array'])
filtered_df['ipa'] = res
filtered_df['ipa_array'] = res_list

100%|██████████| 603978/603978 [15:37<00:00, 644.19it/s]
/tmp/ipykernel_61636/3843269787.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['ipa'] = res
/tmp/ipykernel_61636/3843269787.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['ipa_array'] = res_list


In [22]:
filtered_df.head()

,audio_path,label_path,label,label.transformed,ipa,ipa_array
0,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,그렇게 되면 또 (2월)/(이 월)로 넘어가겠지.\n,그렇게 되면 또 이 월로 넘어가겠지,kɯɾʌkʰɛ dwɛmjʌn t*o i wʌllo nʌmʌɡɑɡɛttɕ*i,"[k, ɯ, ɾ, ʌ, kʰ, ɛ, , d, wɛ, m, jʌ, n, , t*,..."
1,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,뭐/ 기아. 막/ 그런 사람들이 어떻게 더 좋게 살아갈 수 있을까. 이렇게 b/ 평...,뭐 기아 막 그런 사람들이 어떻게 더 좋게 살아갈 수 있을까 이렇게 평상시에도 관심...,mwʌ ɡiɑ mɑk k*ɯɾʌn sɑɾɑmdɯɾi ʌt*ʌkʰɛ dʌ dʑokʰɛ...,"[m, wʌ, , ɡ, i, ɑ, , m, ɑ, k, , k*, ɯ, ɾ, ʌ..."
2,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,"중점적으로 하는 학원인데, b/ 그거를 해서 아이들이 공부하는 까. b/\n",중점적으로 하는 학원인데 그거를 해서 아이들이 공부하는 까,tɕuŋdʑʌmdʑʌɡɯɾo hɑnɯn hɑɡwʌnindɛ ɡɯɡʌɾɯl hɛsʌ ...,"[tɕ, u, ŋ, dʑ, ʌ, m, dʑ, ʌ, ɡ, ɯ, ɾ, o, , h, ..."
3,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,공단은 나도 잘 모르겠다. (4층)/(사 층)인가? n/ 아니 공단.\n,공단은 나도 잘 모르겠다 사 층인가 아니 공단,koŋdɑnɯn nɑdo dʑɑl moɾɯɡɛtt*ɑ sɑ tɕʰɯŋiŋɡɑ ɑni...,"[k, o, ŋ, d, ɑ, n, ɯ, n, , n, ɑ, d, o, , dʑ,..."
4,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,n/ 아까 뭐 만들었는지 얘기해줬잖아.\n,아까 뭐 만들었는지 얘기해줬잖아,ɑk*ɑ mwʌ mɑndɯɾʌnnɯndʑi jɛɡihɛdʑwʌttɕ*ɑnhɑ,"[ɑ, k*, ɑ, , m, wʌ, , m, ɑ, n, d, ɯ, ɾ, ʌ, n..."


In [30]:
filtered_df.columns = ['soundpath', 'labelpath', 'script.text', 'script.text.transformed', 'script.text.ipa', 'script.text.ipa_array']
new_df = filtered_df[['soundpath', 'script.text.ipa', 'script.text.ipa_array', 'script.text.transformed']]

In [31]:
new_df.head()

,soundpath,script.text.ipa,script.text.ipa_array,script.text.transformed
0,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,kɯɾʌkʰɛ dwɛmjʌn t*o i wʌllo nʌmʌɡɑɡɛttɕ*i,"[k, ɯ, ɾ, ʌ, kʰ, ɛ, , d, wɛ, m, jʌ, n, , t*,...",그렇게 되면 또 이 월로 넘어가겠지
1,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,mwʌ ɡiɑ mɑk k*ɯɾʌn sɑɾɑmdɯɾi ʌt*ʌkʰɛ dʌ dʑokʰɛ...,"[m, wʌ, , ɡ, i, ɑ, , m, ɑ, k, , k*, ɯ, ɾ, ʌ...",뭐 기아 막 그런 사람들이 어떻게 더 좋게 살아갈 수 있을까 이렇게 평상시에도 관심...
2,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,tɕuŋdʑʌmdʑʌɡɯɾo hɑnɯn hɑɡwʌnindɛ ɡɯɡʌɾɯl hɛsʌ ...,"[tɕ, u, ŋ, dʑ, ʌ, m, dʑ, ʌ, ɡ, ɯ, ɾ, o, , h, ...",중점적으로 하는 학원인데 그거를 해서 아이들이 공부하는 까
3,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,koŋdɑnɯn nɑdo dʑɑl moɾɯɡɛtt*ɑ sɑ tɕʰɯŋiŋɡɑ ɑni...,"[k, o, ŋ, d, ɑ, n, ɯ, n, , n, ɑ, d, o, , dʑ,...",공단은 나도 잘 모르겠다 사 층인가 아니 공단
4,./data/korean/KsponSpeech_04/KsponSpeech_0373/...,ɑk*ɑ mwʌ mɑndɯɾʌnnɯndʑi jɛɡihɛdʑwʌttɕ*ɑnhɑ,"[ɑ, k*, ɑ, , m, wʌ, , m, ɑ, n, d, ɯ, ɾ, ʌ, n...",아까 뭐 만들었는지 얘기해줬잖아


In [32]:
new_df.to_pickle('./data/korean/preprocessed.pickle')

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
train_df, test_df = train_test_split(
    new_df,
    test_size=0.2,       # test set 비율 (또는 train_size=0.8 사용)
    random_state=42,     # 재현성을 위한 시드
    shuffle=True         # (기본값) 랜덤 셔플
)

In [35]:
train_df.to_pickle('./data/korean/preprocessed_train.pickle')
test_df.to_pickle('./data/korean/preprocessed_test.pickle')